# RAG Pipeline

Onderstaande code is een voorbeeld van een RAG-pipeline met een taalmodel dat lokaal gedraaid kan worden.
Het model in kwestie is een quantized versie van de 4B-parameterer-variant van Qwen3.
Dit model is kleiner dan 3G GB en daarom geschikt om ook op consumer-grade laptops te draaien.
Daarnaast heeft dit model reasoning-capaciteiten (hij "denkt" eerst na voordat hij een antwoord geeft) en is redelijk capabel in Nederlands.

## Imports

Importeer de nodige functionaliteit uit Owlsight voor de RAG-Pipeline.

Documentsearcher: Fungeert als retriever voor het zoeken van relevante stukken tekst op basis van een gebruikersvraag

SemanticTextSplitter: Split tekst op in chunks voor de retriever

DocumentReader: Voor het inlezen en omzetten van tekst van documenten

select_processor_type: Koppelt juiste modelfunctionaliteit vanuit owlsight aan model op basis van modelnaam

In [ ]:
# stappen runnen llama-cpp-python met owlsight:

# uv pip install owlsight

# voor macbook, voer deze stappen extra uit:
# export FORCE_CMAKE=1                             
# export CMAKE_ARGS="-DGGML_METAL=on"

# uv pip install -U llama-cpp-python --no-cache-dir

In [ ]:
from owlsight import DocumentSearcher, DocumentReader, SemanticTextSplitter, select_processor_type
import os

## Lees documenten in

Met de DocumentReader class, gebouwd op Apache Tika, kunnen files in allerlei verschillende extensies naar tekst worden omgezet.
Voor het inlezen van een enkele file, kan de read_file methode worden gebruikt.

Als alternatief kan ook de read_directory methode worden gebruikt, waarmee een complete directory (recursief) kan worden ingelezen.
Dit is handig voor het verwerken van een grote set aan documenten.

In [ ]:
documents_path = "/path/naar/jouw/documenten/hier"

if not os.path.exists(documents_path):
    print(f"path {documents_path} bestaat niet!")

reader = DocumentReader()
if not os.path.isdir(documents_path):
    content = reader.read_file(documents_path)
    documents = {documents_path: content}
else:
    documents = dict(reader.read_directory(documents_path, recursive=True))
    
print(documents.keys())

## Componenten RAG-pipeline 

Een typische RAG-pipeline bestaat, naast een taalmodel, uit twee hoofdcomponenten:

TextSplitter:
Deze component splitst documenten of tekstbestanden in kleinere fragmenten (chunks), 
vaak op basis van paragrafen, zinnen, of tokens.
In Owlsight wordt exclusief gebruik gemaakt van de Sentence-Transformers library.


Retriever (met Encoder):
Elk fragment wordt omgezet in een vector (numerieke representatie) met behulp van een encoder, 
meestal een embedding-model. Deze vectoren worden vervolgens opgeslagen. 
Meestal gebeurt dit in een vector database (zoals FAISS of Weaviate).
In Owlsight worden de vectoren opgeslagen met pickle.

Bij een gebruikersvraag wordt dezelfde encoder gebruikt om ook de vraag naar een vector om te zetten, waarna de meest relevante fragmenten (dichtstbijzijnde vectoren) worden opgehaald.

In [ ]:
embedding_model = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
text_splitter=SemanticTextSplitter(model_name=embedding_model, target_chunk_length=500)
searcher = DocumentSearcher(documents, 
                            text_splitter=text_splitter,
                            sentence_transformer_model = embedding_model,
                            cache_dir="documents", 
                           cache_dir_suffix="rag_pipeline_example")

In [ ]:
# Laat dataframe zien met relevante chunks en scores
# aggregated_score is een combinatie van tfidf en het embedding model
# het voordeel van het gebruiken van tfidf is dat dit een snelle keysearch is die ook woorden meeneemt waar een embedding model niet per se
# op is getraind. Als in een query een "moderner" woord als "fatbike" staat, wordt deze alsnog meegenomen.
retrieved_chunks = searcher.search("Stel je vraag over je document hier", top_k=20, as_context=False)
retrieved_chunks.head()

# Taalmodel

Nu laden we het taalmodel in. Met de select_processor_type functie, kunnen we gelijk het juiste model inladen.
In Owlsight gebeurt dit door een abstractielaag, genaamd TextGenerationProcessor.
Model-specifieke parameters zijn uitgedrukt met dubbele underscore "__", bijvoorbeeld gguf__filename.

In [ ]:
model_id = "unsloth/Qwen3-4B-GGUF"

# koppel juiste processor aan model op basis van model_id
model_processor_cls = select_processor_type(model_id)

model_processor = model_processor_cls(
    model_id=model_id,
    apply_chat_history = False,
    system_prompt="Je bent een behulpzame assistent. Als een vraag niet beantwoord kan worden op basis van de context, zeg dan: 'Ik kan de vraag op basis van de context niet beantwoorden.'",
    gguf__verbose=False,
    gguf__n_gpu_layers=32,
    gguf__n_ctx=4000,
    gguf__filename="Qwen3-4B-UD-Q5_K_XL.gguf")

# Chatbot met RAG

Nu hebben we alle elementen voor een chatbot met RAG-pipeline.
In ons voorbeeld is dit een "instruct"-bot, waarbij de chathistorie bij elke nieuwe vraag of query wordt ververst.
Dit zodat we niet opnieuw de nieuwe chunks uit een vervolgvraag meegeven.
Met behulp van de searcher (retriever), geven we elke keer de 5 meest relevante documenten mee op basis van de userquery.

In [ ]:
from IPython.display import clear_output

top_k = 5

model_processor.clear_history()
while True:
    query = input("(Typ 'stop' voor stoppen, 'clear' voor nieuwe chat)\nJij: ")
    if query.lower() in ["stop", "exit", "quit"]:
        print("Chatbot: Tot ziens!")
        break
    # elif query.lower() == "clear":
    #     model_processor.clear_history()
    #     clear_output(wait=True)  # wist de output van notebookcel
    #     print("Chatgeschiedenis is gewist en uitvoer is ververst.")
    #     continue

    # Zoek relevante context
    try:
        context=searcher.search(query, top_k=top_k, as_context=True)
        seperator = "-" * 50
        print(f"Gebruikte context:\n{seperator}\n{context}\n{seperator}")
    except Exception as e:
        print(f"Fout bij zoeken: {e}")
        context = ""

    # voeg context toe aan de query
    full_query = query + "\n\nGebruik onderstaande tekst om je antwoord te geven:\n" + context

    # Genereer antwoord
    try:
        print("Chatbot:")
        response = model_processor.generate(full_query, max_new_tokens=800)
        model_processor.clear_history()
    except Exception as e:
        response = f"Fout bij genereren: {e}"